# JAMA revision round 4

In [0]:
!pip install lifelines

## Import packages

In [0]:
import pandas as pd, numpy as np, re
from numpy import exp, mean
from fancyimpute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from lifelines import CoxPHFitter, AalenJohansenFitter

In [0]:
from google.colab import drive
drive.mount('/content/Drive')

## **Read data**

In [0]:
# whole cohort (N=1,478,506)
d = pd.read_csv('Drive/My Drive/facility/Fortable3mi_ld_dec_censored.csv')
d['rucc_rural'] = np.where(d.RUCC_2013.isin([1, 2, 3]), 0, (np.where(d.RUCC_2013.isna(), np.nan, 1)))
d = d.drop(['RUCC_2013'], axis=1)
# d.loc[d['wl_time']>24, 'wl']=0
# d.loc[d['wl_time']>24, 'wl_time']=24
# d.loc[d['ld_time']>24, 'livingd']=0
# d.loc[d['ld_time']>24, 'ld_time']=24
# d.loc[d['dec_time']>24, 'deceasedt']=0
# d.loc[d['dec_time']>24, 'dec_time']=24
# d.to_csv('drive/My Drive/facility/jama_round4.csv', index=False)

#**Multiple imputation**


##**Random forest imputation for categorical**

In [0]:
for var in ['dialysis_mod1', 'insurance_esrd', 'rucc_rural', 'nephcare_cat2']:
    pred = ['sex_new', 'age_cat', 'race_new', var]
    imputer = IterativeImputer(n_iter=1, random_state=7, predictor=RandomForestClassifier(n_estimators=10))
    imputed = pd.DataFrame(imputer.fit_transform(d[pred]), columns=pred)
    d = d.drop(var, axis=1).join(imputed[var])


##**Bayesian Ridge linear imputation for continuous**


In [0]:
for var in ['Hospitalization_Rate_facility', 'Mortality_Rate_Facility', 'NEAR_DIST']:
    completed = []
    for i in range(5):
        pred = ['sex_new', 'age_cat', 'race_new', var]
        imputer = IterativeImputer(n_iter=5, sample_posterior=True, random_state=i)
        completed.append(imputer.fit_transform(d[pred]))
    completed_mean = np.mean(completed, axis=0)
    imputed = pd.DataFrame(completed_mean, columns=pred)
    if var == 'NEAR_DIST':
        m = imputed[imputed.NEAR_DIST > 0].NEAR_DIST.mean()
        imputed.NEAR_DIST = np.where(imputed.NEAR_DIST < 0, m, imputed.NEAR_DIST)
    d = d.drop(var, axis=1).join(imputed[var])

#**Create cohort for Cox model**

**1) dummy code and order levels based on table**

**2) drop unneeded variables**

In [0]:
# standard cohort
PH_data = d[['PROVUSRD', 'chain_class2', 'for_profit', 'sex_new', 'age_cat', 'race_new', 'dialysis_mod1', 'esrd_cause', 'bmi_35',
                 'ashd_new', 'chf',	'other_cardiac', 'cva_new',	'pvasc_new', 'hypertension', 'diabetes', 'copd_new',
                 'smoke_new', 'cancer_new', 'insurance_esrd', 'PATTXOP_MEDUNFITn','nephcare_cat2', 'profit_txc', 'profit_hosp',
                 'network_us_region_dfr', 'NEAR_DIST', 'rucc_rural', 'wl', 'wl_time', 'livingd', 'ld_time', 'deceasedt', 'dec_time']]
PH_data = PH_data.join(pd.get_dummies(PH_data.dialysis_mod1, prefix='dialysis_mod1', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.insurance_esrd, [3, 2, 1, 4, 5], True), prefix='insurance_esrd', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.age_cat, [5, 1, 2, 3, 4, 6], True), prefix='age_cat', drop_first=True)) # delete category "6" for ideal cohort!
PH_data = PH_data.join(pd.get_dummies(PH_data.race_new, prefix='race_new', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.esrd_cause, prefix='esrd_cause', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.network_us_region_dfr, prefix='network_us_region_dfr', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.chain_class2, [6,5,2,1,3,4], True), prefix='chain_class2', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.profit_txc, prefix='profit_txc', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.profit_hosp, prefix='profit_hosp', drop_first=True))
PH_data = PH_data.drop(['dialysis_mod1', 'esrd_cause', 'age_cat', 'race_new', 'chain_class2', 'insurance_esrd', 'network_us_region_dfr', 'profit_txc', 'profit_hosp'], axis=1)

# ideal cohort
ideal = d[(d.INC_AGE < 66) &
  (d.pvasc_new == 0) &
  (d.chf == 0) &
  (d.cva_new == 0) &
  (d.PATTXOP_MEDUNFITn == 0)].reset_index(drop=True)

PH_data_ideal = ideal[['PROVUSRD', 'chain_class2', 'for_profit', 'sex_new', 'age_cat', 'race_new', 'dialysis_mod1', 'esrd_cause', 'bmi_35',
                 'ashd_new', 'chf',	'other_cardiac', 'cva_new',	'pvasc_new', 'hypertension', 'diabetes', 'copd_new',
                 'smoke_new', 'cancer_new', 'insurance_esrd', 'PATTXOP_MEDUNFITn','nephcare_cat2',
                 'network_us_region_dfr', 'NEAR_DIST', 'rucc_rural', 'wl', 'wl_time', 'livingd', 'ld_time', 'deceasedt', 'dec_time']]
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.dialysis_mod1, prefix='dialysis_mod1', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.insurance_esrd, [3, 2, 1, 4, 5], True), prefix='insurance_esrd', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.age_cat, [5, 1, 2, 3, 4], True), prefix='age_cat', drop_first=True)) # delete category "6" for ideal cohort!
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.race_new, prefix='race_new', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.esrd_cause, prefix='esrd_cause', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.network_us_region_dfr, prefix='network_us_region_dfr', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.chain_class2, [6,5,2,1,3,4], True), prefix='chain_class2', drop_first=True))
PH_data_ideal = PH_data_ideal.drop(['dialysis_mod1', 'esrd_cause', 'age_cat', 'race_new', 'chain_class2', 'insurance_esrd', 'network_us_region_dfr'], axis=1)


##**Table 2**

In [0]:
varlist=['age_cat',
 'sex_new',
 'race_new',
 'insurance_esrd',
 'esrd_cause',
 'dialysis_mod1',
 'bmi_35',
 'chf',
 'ashd_new',
 'other_cardiac',
 'cva_new',
 'pvasc_new',
 'hypertension',
 'diabetes',
 'copd_new',
 'smoke_new',
 'cancer_new',
 'nephcare_cat2',
 'PATTXOP_MEDUNFITn',
 'network_us_region_dfr',
 'rucc_rural',
 'NEAR_DIST']


In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in varlist:
    crude = '|'.join([exposure, time, status])
    cph.fit(PH_data.filter(regex=crude), duration_col=time, event_col=status, step_size=0.5)
    print(round(pd.concat([exp(cph.hazards_).rename('HR'), exp(cph.confidence_intervals_)], 1), 2))

In [0]:
for case, time in zip(['wl', 'livingd', 'deceasedt'], ['wl_time', 'ld_time', 'dec_time']):
  tmp = d
  count = tmp.groupby('chain_class2')[case].sum().map('{:,}'.format)
  pyr = (tmp.groupby('chain_class2')[time].sum()/12).map(' /{:,.0f}'.format)
  print(count+pyr)
  count1 = tmp.groupby('for_profit')[case].sum().map('{:,}'.format)
  pyr1 = (tmp.groupby('for_profit')[time].sum()/12).map(' /{:,.0f}'.format)
  print(count1+pyr1)


##**Table 3**

In [0]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
import statsmodels.formula.api as smf
import statsmodels.api as sm
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
utils = importr('utils')
utils.install_packages('msm')
msm = importr('msm')


In [17]:
# Truncate follow-up time
def truncate(t):
    e = d.copy()
    e.loc[e['wl_time'] > t, 'wl']=0
    e.loc[e['wl_time'] > t, 'wl_time']=t
    e.loc[e['ld_time'] > t, 'livingd']=0
    e.loc[e['ld_time'] > t, 'ld_time']=t
    e.loc[e['dec_time'] > t, 'deceasedt']=0
    e.loc[e['dec_time'] > t, 'dec_time']=t
    return e

# dataset with aggregate sum for poisson model
for event, time in zip(['wl','livingd', 'deceasedt'],['wl_time', 'ld_time', 'dec_time']):
	for cutoff in [12, 36, 60]:
		poissonset = truncate(cutoff).groupby(['for_profit', 'age_cat', 'sex_new', 'race_new'], as_index=False)[['wl', 'wl_time', 'livingd', 'ld_time', 'dec_time', 'deceasedt']].sum()
		poissonset['age_cat'] = pd.Categorical(poissonset['age_cat'], ordered=True, categories=[5, 1, 2, 3, 4, 6])
		poissonset['race_new'] = pd.Categorical(poissonset['race_new'], ordered=True, categories=[1, 2, 3, 4])

		# poisson regression
		poisson_model = smf.glm(event+'~for_profit+sex_new+age_cat+race_new',
								data=poissonset,
								family=sm.families.Poisson(),
								offset=np.log(poissonset[time]/12)).fit(method='newton')

		# extract coefficient
		b0 = poisson_model.params['Intercept']
		b1 = poisson_model.params['for_profit']

		# calculate rate difference and standard error
		effect = np.exp(b0 + b1) - np.exp(b0)
		vcov = poisson_model.cov_params().loc[['Intercept', 'for_profit'], ['Intercept', 'for_profit']]
		se = msm.deltamethod(ro.Formula('~exp(x1+x2)-exp(x1)'), FloatVector([b0, b1]), ro.r.matrix(vcov.values, nrow=2, ncol=2))
		print("%s %g-months for_profit vs. non_profit: %.2f (%.2f, %.2f) " %(event, cutoff, effect*100, (effect-1.96*float(np.array(se)))*100, (effect+1.96*float(np.array(se)))*100))

wl 12-months for_profit vs. non_profit: -9.62 (-9.89, -9.36) 
wl 36-months for_profit vs. non_profit: -6.75 (-6.90, -6.61) 
wl 60-months for_profit vs. non_profit: -5.38 (-5.49, -5.27) 
livingd 12-months for_profit vs. non_profit: -0.74 (-0.80, -0.67) 
livingd 36-months for_profit vs. non_profit: -0.76 (-0.80, -0.71) 
livingd 60-months for_profit vs. non_profit: -0.64 (-0.67, -0.60) 
deceasedt 12-months for_profit vs. non_profit: -0.23 (-0.26, -0.19) 
deceasedt 36-months for_profit vs. non_profit: -0.80 (-0.84, -0.75) 
deceasedt 60-months for_profit vs. non_profit: -1.08 (-1.13, -1.03) 


In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['chain', 'for_profit']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn' #PATTXOP_MEDUNFITn'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model_'+str(i))
      cph.fit(PH_data.filter(regex=model), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))




##**Figure 2**

In [0]:
d['FIRST_SE'] = pd.to_datetime(d['FIRST_SE'], format='%m/%d/%Y')
fig2 = pd.DataFrame()

for event, time in zip(['wl', 'livingd', 'deceasedt'], ['wl_time', 'ld_time', 'dec_time']):
  print('-'*40, event, '-'*40)
  for exposure in ['chain_class2', 'for_profit']:
    print('\n', exposure)
    fig2 = pd.DataFrame()
    for year in range(2001, 2016, 2):
        num = d[(d['FIRST_SE']>=str(year)+'/01/01') & (d['FIRST_SE']<=str(year+1)+'/12/31')].groupby(exposure)[event].sum()
        pyr = d[(d['FIRST_SE']>=str(year)+'/01/01') & (d['FIRST_SE']<=str(year+1)+'/12/31')].groupby(exposure)[time].sum()/12
        rate = (num/pyr*100).rename(str(year)+'-'+"{0:0>2}".format(year+1-2000))
        fig2 = pd.concat([fig2, rate], 1)
    print(fig2.applymap('{:.2f}'.format))


##**Supplemental Table 3 (ideal cohort)**

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['chain', 'for_profit']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural' #PATTXOP_MEDUNFITn'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model'+str(i))
      cph.fit(PH_data_ideal.filter(regex=model), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))



##**Supplemental Table 4**

In [0]:
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for r, region in enumerate(['Northeast', 'South', 'Midwest', 'West']):
    print('\n', region)
    for exposure in ['chain', 'for_profit']:
      crude = '|'.join([exposure, time, status])
      model1 = crude + '|sex_new|age_cat|race_new'
      model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                        'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
      model3 = model2 + '|insurance_esrd|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn' #network_us_region_dfr'
      cph.fit(PH_data[d['network_us_region_dfr']==r].filter(regex=model3), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))


##Supplemental Table 5 & 6

In [0]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['profit_txc', 'profit_hosp']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                      'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn|network_us_region_dfr' #network_us_region_dfr'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model_'+str(i))
      cph.fit(PH_data.filter(regex=model), duration_col=time, event_col=status, step_size=0.2)
      print(round(pd.concat([exp(cph.hazards_[cph.hazards_.index.str.contains(exposure)]).rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))